# Regenerate all_models.json

This script regenerates the `all_models.json` file for each prompt folder by combining all individual model JSON files.

In [1]:
import json
import os
from pathlib import Path

def regenerate_all_models_json(prompt_folder_path, output_folder_path=None):
    """
    Regenerates the all_models.json file for a given prompt folder.
    
    Args:
        prompt_folder_path: Path to the prompt folder (e.g., 'Straico/Results/Prompt 1')
        output_folder_path: Optional path where to save the output (defaults to prompt_folder_path)
    """
    prompt_folder = Path(prompt_folder_path)
    
    if not prompt_folder.exists():
        print(f"Error: Folder {prompt_folder} does not exist")
        return
    
    all_models_data = {}
    
    # Iterate through all subdirectories in the prompt folder
    for subdir in sorted(prompt_folder.iterdir()):
        # Skip the all_models.json file itself
        if subdir.name == "all_models.json" or not subdir.is_dir():
            continue
        
        # Look for the JSON file inside the subdirectory
        json_file = subdir / f"{subdir.name}.json"
        
        if json_file.exists():
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    model_data = json.load(f)
                
                # Extract the model name from the first entry
                if model_data and len(model_data) > 0:
                    model_name = model_data[0].get('model', subdir.name)
                    all_models_data[model_name] = model_data
                    print(f"✓ Loaded {subdir.name}: {len(model_data)} entries")
                else:
                    print(f"⚠ Warning: {json_file} is empty")
                    
            except json.JSONDecodeError as e:
                print(f"✗ Error reading {json_file}: {e}")
            except Exception as e:
                print(f"✗ Unexpected error with {json_file}: {e}")
        else:
            print(f"⚠ Warning: {json_file} not found")
    
    # Determine output folder
    output_folder = Path(output_folder_path) if output_folder_path else prompt_folder
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Write the combined data to all_models.json
    output_file = output_folder / "all_models.json"
    
    if all_models_data:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(all_models_data, f, ensure_ascii=False, indent=2)
        
        total_entries = sum(len(entries) for entries in all_models_data.values())
        print(f"\n✓ Successfully created {output_file}")
        print(f"  Total models: {len(all_models_data)}")
        print(f"  Total entries: {total_entries}")
    else:
        print("\n✗ No data found to write to all_models.json")

def process_azure_results(azure_folder_path, output_folder_path):
    """
    Process Azure results and convert to all_models.json format.
    
    Args:
        azure_folder_path: Path to Azure prompt folder
        output_folder_path: Path where to save the combined results
    """
    azure_folder = Path(azure_folder_path)
    
    if not azure_folder.exists():
        print(f"⚠ Warning: Azure folder {azure_folder} does not exist")
        return
    
    all_models_data = {}
    
    # Look for JSON files in the Azure results folder
    for json_file in sorted(azure_folder.glob("*.json")):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                model_data = json.load(f)
            
            # Extract model name from the first entry or filename
            if model_data and len(model_data) > 0:
                model_name = model_data[0].get('model', json_file.stem)
                all_models_data[model_name] = model_data
                print(f"✓ Loaded Azure {json_file.name}: {len(model_data)} entries")
            else:
                print(f"⚠ Warning: {json_file} is empty")
                
        except json.JSONDecodeError as e:
            print(f"✗ Error reading {json_file}: {e}")
        except Exception as e:
            print(f"✗ Unexpected error with {json_file}: {e}")
    
    return all_models_data

def combine_straico_and_azure(prompt_number):
    """
    Combines results from both Straico and Azure for a given prompt.
    
    Args:
        prompt_number: The prompt number (1, 2, 3, etc.)
    """
    print(f"\n{'='*60}")
    print(f"Processing Prompt {prompt_number}")
    print(f"{'='*60}")
    
    # Define paths
    base_path = Path("/Users/sbobi/Downloads/Modismos Papers/LLMs_APIs")
    straico_folder = base_path / f"Straico/Results/Prompt {prompt_number}"
    azure_folder = base_path / f"Azure/Results/Prompt {prompt_number}"
    output_folder = base_path / f"Results/Prompt {prompt_number}"
    
    # Create output folder
    output_folder.mkdir(parents=True, exist_ok=True)
    
    combined_data = {}
    
    # Process Straico results
    print("\n--- Processing Straico Results ---")
    if straico_folder.exists():
        for subdir in sorted(straico_folder.iterdir()):
            if subdir.name == "all_models.json" or not subdir.is_dir():
                continue
            
            json_file = subdir / f"{subdir.name}.json"
            
            if json_file.exists():
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        model_data = json.load(f)
                    
                    if model_data and len(model_data) > 0:
                        model_name = model_data[0].get('model', subdir.name)
                        combined_data[model_name] = model_data
                        print(f"✓ Loaded Straico {subdir.name}: {len(model_data)} entries")
                except Exception as e:
                    print(f"✗ Error with {json_file}: {e}")
    else:
        print(f"⚠ Straico folder not found: {straico_folder}")
    
    # Process Azure results
    print("\n--- Processing Azure Results ---")
    if azure_folder.exists():
        for json_file in sorted(azure_folder.glob("*.json")):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    model_data = json.load(f)
                
                if model_data and len(model_data) > 0:
                    model_name = model_data[0].get('model', json_file.stem)
                    combined_data[model_name] = model_data
                    print(f"✓ Loaded Azure {json_file.name}: {len(model_data)} entries")
            except Exception as e:
                print(f"✗ Error with {json_file}: {e}")
    else:
        print(f"⚠ Azure folder not found: {azure_folder}")
    
    # Save combined results
    if combined_data:
        output_file = output_folder / "all_models.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(combined_data, f, ensure_ascii=False, indent=2)
        
        total_entries = sum(len(entries) for entries in combined_data.values())
        print(f"\n{'='*60}")
        print(f"✓ Successfully saved combined results to:")
        print(f"  {output_file}")
        print(f"  Total models: {len(combined_data)}")
        print(f"  Total entries: {total_entries}")
        print(f"{'='*60}")
    else:
        print("\n✗ No data found to combine")

In [2]:
# To process all prompt folders automatically (Prompts 1, 2, and 3):
for prompt_num in [1, 2, 3]:
    combine_straico_and_azure(prompt_num)


Processing Prompt 1

--- Processing Straico Results ---
✓ Loaded Straico amazon_nova_lite_v1: 6533 entries
✓ Loaded Straico amazon_nova_micro_v1: 6533 entries
✓ Loaded Straico anthropic_claude_sonnet_4: 6533 entries
✓ Loaded Straico cohere_command_r_08_2024: 6533 entries
✓ Loaded Straico deepseek_deepseek_chat_v3_1: 6533 entries
✓ Loaded Straico google_gemini_2_5_flash: 6533 entries
✓ Loaded Straico google_gemma_2_27b_it: 6533 entries
✓ Loaded Straico meta_llama_llama_3_1_405b_instruct: 6533 entries
✓ Loaded Straico meta_llama_llama_3_3_70b_instruct: 6533 entries
✓ Loaded Straico meta_llama_llama_4_maverick: 6533 entries
✓ Loaded Straico microsoft_phi_4: 6533 entries
✓ Loaded Straico microsoft_wizardlm_2_8x22b: 6533 entries
✓ Loaded Straico mistralai_mistral_medium_3: 6533 entries
✓ Loaded Straico mistralai_mixtral_8x7b_instruct: 6533 entries
✓ Loaded Straico moonshotai_kimi_k2_0905: 6533 entries
✓ Loaded Straico openai_gpt_4_1: 6533 entries
✓ Loaded Straico openai_o1_mini: 6533 entri